In [2]:
from binance.um_futures import UMFutures
# time.mktime(time.strptime('2018-01-01 00:00:00'))
# 将时间字符串转为时间戳

um_futures_client = UMFutures()
instruments = um_futures_client.exchange_info()['symbols']
instruments = [i['pair'] for i in instruments if i['contractType'] == 'PERPETUAL' and i['pair'].endswith('USDT')]
# instruments


In [3]:
import time
import dolphindb as ddb
import pandas as pd

def get_start_time(instrument):
    sess = ddb.session(host='localhost', port=8848, userid='admin', password='123456')
    df = sess.run(f"select max(trade_time) as t from loadTable('dfs://crypto_db', 'k_hour') where instrument = '{instrument}'")
    sess.close()
    if pd.isna(df['t'].iloc[0]):
        start_time = int(time.mktime(time.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S'))) * 1000
    else:
        print(df)
        start_time = int(df.iloc[0]['t'].tz_localize('Asia/Shanghai').timestamp()) * 1000
    return start_time

In [4]:
from tqdm import tqdm
import dolphindb as ddb
import pandas as pd


sess = ddb.session(host='localhost', port=8848, userid='admin', password='123456', enableASYNC=True)
def insert_data(instrument):
    start_time = get_start_time(instrument)
    # 每小时的时间戳
    # interval = 3600 * 1000
    tqdm_num = int(time.time()) * 1000 - start_time
    with tqdm(total=int(tqdm_num)) as pbar:
        while start_time <= int(time.time()) * 1000:
            data = um_futures_client.klines(instrument, "1h", startTime=start_time, limit=499)
            df = pd.DataFrame(data, columns=['trade_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'amount', 'trade_num', 'acitve_volume', 'active_amount', 'drop'])
            df['instrument'] = instrument
            df['exchange'] = 'BINANCE'
            df = df.drop(columns=['close_time', 'drop'])
            df['high'] = df['high'].astype(float)
            df['open'] = df['open'].astype(float)
            df['low'] = df['low'].astype(float)
            df['close'] = df['close'].astype(float)
            df['volume'] = df['volume'].astype(float)
            df['acitve_volume'] = df['acitve_volume'].astype(float)
            df['active_amount'] = df['active_amount'].astype(float)
            df['amount'] = df['amount'].astype(float)
            # 时间差小于1小时
            # print((df['trade_time'].iloc[-1] - start_time) / 3600 / 1000)
            interval = (df['trade_time'].iloc[-1] - start_time)
            if df['trade_time'].iloc[-1] - start_time < 3600 * 1000 * 498:
                start_time = int(time.time()) * 1200
            else:
                start_time = df['trade_time'].iloc[-1] + 3600 * 1000
            df['trade_time'] = df['trade_time'] / 1000
            df['trade_time'] = pd.to_datetime(df['trade_time'], unit='s')
            df['trade_time'] = df['trade_time'] + pd.Timedelta(hours=8)
            df = df[['instrument', 'exchange', 'trade_time', 'open', 'close', 'high', 'low', 'volume', 'amount', 'trade_num', 'acitve_volume', 'active_amount']]
            sess.run(f"append!{{loadTable('dfs://crypto_db', `k_hour)}}", df)
            # appender = ddb.tableAppender(dbPath="dfs://crypto_db", tableName="k_hour", ddbSession=sess)
            # num = appender.append(df)
            pbar.update(interval)
        pbar.update(1)


for instrument in instruments:
    print(instrument)
    insert_data(instrument)
    print("done")
sess.close()

BTCUSDT
                    t
0 2023-11-15 22:00:00


100%|█████████▉| 8672400001/8688989000 [00:01<00:00, 6779745171.39it/s]


done
ETHUSDT
                    t
0 2023-11-15 22:00:00


100%|█████████▉| 8672400001/8688990000 [00:01<00:00, 7906209262.57it/s]


done
BCHUSDT
                    t
0 2023-11-15 22:00:00


...

done
MAVIAUSDT


100%|█████████▉| 194007600001/194010141000 [00:00<00:00, 1083779869176.35it/s]


done
GLMUSDT


100%|█████████▉| 194007600001/194010141000 [00:00<00:00, 1091560708216.14it/s]

done
